In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sleeploader
import tools
import matplotlib
import time
import glob
import os,sys
import traceback
from scipy.io import loadmat
import mne
import pprint
from collections import Counter
import _pickle as cPickle

import keras
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Conv1D, MaxPooling1D
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.utils import np_utils

pp = pprint.PrettyPrinter(indent=4)

label_sets = [
    {'sets':{'Sleep_stage_W':0,
                          'Sleep_stage_R':1,
                          'Sleep_stage_1':2,
                          'Sleep_stage_2':3,
                          'Sleep_stage_3':4,
                          'Sleep_stage_4':5,
                          'Sleep_stage_?':6,
                          'Movement_time':7,
                         },
    'num_label':8},
    {'sets':{'Sleep_stage_W':0,
                          'Sleep_stage_R':1,
                          'Sleep_stage_1':2,
                          'Sleep_stage_2':2,
                          'Sleep_stage_3':2,
                          'Sleep_stage_4':2,
                          'Sleep_stage_?':0,
                          'Movement_time':0,
                         },
    'num_label':3},
]

In [30]:
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_CNN_trained_model.h5'

time_window = 500
label_set = 1
n_input = time_window
input_len = 1
n_hiddens = [300,100]
batch_size = 128
n_output = label_sets[label_set]['num_label']
epochs = 100

file_name = "tw_{}_ls_{}_il_{}_bs_{}.pkl".format(time_window,label_set,input_len,batch_size)
if os.path.exists(file_name):
    print('exists {}'.format(file_name))
    with open(file_name,'rb') as f:
        DD = cPickle.load(f)
    print('load Done')

print("length of Datasets: {}".format(len(DD)))
Dt = [[],[]]
for D in DD:
    x_train, y_train, x_test, y_test = D
    x_train = np.transpose(x_train, (0, 2, 1))
    x_test = np.transpose(x_test, (0, 2, 1))
    y_train = np.repeat(y_train, 10,axis=1)
    y_test = np.repeat(y_test, 10,axis=1)
    Dt[0].append(np.concatenate((x_train,x_test),axis=0))
    Dt[1].append(np.concatenate((y_train,y_test),axis=0))

test_rate = 0.1
x_train = np.concatenate(Dt[0],axis=0)
y_train = np.concatenate(Dt[1],axis=0)
ind = int(test_rate*x_train.shape[0])
x_test = x_train[-ind:]
y_test = y_train[-ind:]
x_train = x_train[:-ind]
y_train = y_train[:-ind]
print('x_train: {}'.format(x_train.shape))
print('x_test: {}'.format(x_test.shape))
print('y_train: {}'.format(y_train.shape))
print('y_test: {}'.format(y_test.shape))

exists tw_500_ls_1_il_1_bs_128.pkl
load Done
length of Datasets: 6
x_train: (88935, 500, 1)
x_test: (9881, 500, 1)
y_train: (88935, 10, 3)
y_test: (9881, 10, 3)


In [31]:
def model_1():
    model = Sequential()
    model.add(Conv1D(128, 50, strides=5, activation='relu',input_shape=(time_window,1)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Conv1D(256, 5, strides=1, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    
    model.add(MaxPooling1D(2))
    
    model.add(Conv1D(300, 5, strides=2,  activation='relu'))
    
    model.add(MaxPooling1D(2))
    
    model.add(Dense(1500))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(1500))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))    
    model.add(Dense(n_output))
    model.add(Activation('softmax'))
    return model

def model_2(): # for image classification
    model = Sequential()
    model.add(Conv1D(128, (3, 3), padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    return model

model = model_1()
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_25 (Conv1D)           (None, 91, 128)           6528      
_________________________________________________________________
batch_normalization_27 (Batc (None, 91, 128)           512       
_________________________________________________________________
dropout_27 (Dropout)         (None, 91, 128)           0         
_________________________________________________________________
conv1d_26 (Conv1D)           (None, 87, 256)           164096    
_________________________________________________________________
batch_normalization_28 (Batc (None, 87, 256)           1024      
_________________________________________________________________
dropout_28 (Dropout)         (None, 87, 256)           0         
_________________________________________________________________
max_pooling1d_15 (MaxPooling (None, 43, 256)           0         
__________

In [32]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

# x_train = x_train.astype('float32')
# x_test = x_test.astype('float32')
# x_train /= 255
# x_test /= 255

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True)

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Train on 88935 samples, validate on 9881 samples
Epoch 1/100
88935/88935 [==============================] - 878s 10ms/step - loss: 0.7509 - acc: 0.7161 - val_loss: 0.5297 - val_acc: 0.8299
Epoch 2/100
88935/88935 [==============================] - 891s 10ms/step - loss: 0.3867 - acc: 0.8570 - val_loss: 0.5701 - val_acc: 0.7949
Epoch 3/100
88935/88935 [==============================] - 780s 9ms/step - loss: 0.3395 - acc: 0.8709 - val_loss: 0.4426 - val_acc: 0.8401
Epoch 4/100
88935/88935 [==============================] - 492s 6ms/step - loss: 0.3132 - acc: 0.8795 - val_loss: 0.3744 - val_acc: 0.8493
Epoch 5/100
88935/88935 [==============================] - 492s 6ms/step - loss: 0.2981 - acc: 0.8846 - val_loss: 0.3838 - val_acc: 0.8472
Epoch 6/100
88935/88935 [==============================] - 494s 6ms/step - loss: 0.2886 - acc: 0.8880 - val_loss: 0.3849 - val_acc: 0.8481
Epoch 7/100
88935/88935 [==============================] - 492s 6ms/step - loss: 0.2797 - acc: 0.8909 - val_loss: 0

KeyboardInterrupt: 

In [ ]:
range(3)[1:-1:2]